In [1]:
import sys
import os
import pandas as pd
from pyspark.sql import SparkSession

# --- Célula 1: Configuração do Ambiente e Início da Sessão Spark ---

# Define a raiz do projeto (sobe um nível a partir da pasta 'notebooks')
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Adiciona a pasta 'src' ao caminho de busca do Python para que possamos importar nossos scripts
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

print(f"Caminho do projeto configurado para: {project_root}")

Caminho do projeto configurado para: c:\Users\manto\OneDrive\Documentos\MeusProjetos\Big-Data-forecast


In [2]:
# Inicia a sessão Spark
# spark = SparkSession.builder.appName("NotebookFeatureEngineering").getOrCreate()

import os
os.environ['SPARK_LOCAL_IP'] = '127.0.0.1'
os.environ['SPARK_DRIVER_HOST'] = '127.0.0.1'

spark = SparkSession.builder \
    .appName("NotebookFeatureEngineering") \
    .config("spark.driver.port", "0") \
    .getOrCreate()

print("\nSessão Spark iniciada com sucesso.")


Sessão Spark iniciada com sucesso.


In [3]:
from _02_feature_engineering import run_feature_engineering_pipeline

# Define o caminho para o DataFrame consolidado
input_path = os.path.join(project_root, "data", "processed", "base_consolidada.parquet")

if not os.path.exists(input_path):
    print(f"\nErro: Arquivo '{input_path}' não encontrado. Por favor, execute o 01_load_data.py primeiro.")
else:
    # 1. Carrega o DataFrame consolidado
    print(f"\nCarregando DataFrame de '{input_path}'...")
    df_consolidado = spark.read.parquet(input_path)

    filtered_final_df = df_consolidado.filter(
        col("internal_store_id") == "1048244969886449059"
    ).orderBy("transaction_date")

    # 2. Executa a função de engenharia de features
    print("Executando o pipeline de engenharia de features...")
    df_com_features = run_feature_engineering_pipeline(df_consolidado, spark)

    # 3. Exibe o resultado e salva o arquivo
    if df_com_features:
        print("\nProcesso de engenharia de features concluído.")
        print("Schema do DataFrame final:")
        df_com_features.printSchema()

        output_path = os.path.join(project_root, "data", "processed", "base_com_features.parquet")
        print(f"\nSalvando o DataFrame final em '{output_path}'...")
        df_com_features.write.mode("overwrite").parquet(output_path)
        print("Arquivo salvo com sucesso.")
    
    else:
        print("A execução do pipeline não retornou um DataFrame válido.")
    
# spark.stop()


Carregando DataFrame de 'c:\Users\manto\OneDrive\Documentos\MeusProjetos\Big-Data-forecast\data\processed\base_consolidada.parquet'...
Executando o pipeline de engenharia de features...
Criando features de tempo (semana do ano e semana do mês)...
Criando rank sequencial baseado na ordem das semanas...
Calculando features defasadas...

Processo de engenharia de features concluído.
Schema do DataFrame final:
root
 |-- internal_store_id: string (nullable = true)
 |-- internal_product_id: string (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- distributor_id: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- subcategoria: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- fabricante: string (nullable = true)
 |-- tipos: string (nullable = true)
 |-- label: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- Vendas_Semanais: double (nullable = false)
 |-- gross_value: double (nullable = false)
 |-- net_value: doubl

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "c:\Users\manto\OneDrive\Documentos\MeusProjetos\Big-Data-forecast\.venv\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Users\manto\AppData\Local\Programs\Python\Python310\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\manto\OneDrive\Documentos\MeusProjetos\Big-Data-forecast\.venv\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\manto\OneDrive\Documentos\MeusProjetos\Big-Data-forecast\.venv\lib\site-packages\py4j\clientserver.py", line 539, in send_command
    raise Py4JNetworkError(

ConnectionRefusedError: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente